In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import seaborn as sns
sns.set(style="darkgrid")

# 1. Chargement et présentation des jeux de données

## 1.1 Liste des fichiers en entrées:

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

## 1.2 Chargement des jeux de données

* __zoo.csv__ contient les premières données sur les animaux.

In [ ]:
df_zoo = pd.read_csv("../input/zoo-animal-classification/zoo.csv")
df_zoo.head()

In [ ]:
df_zoo.shape

Chaque animal est définie par un nom (*animal_name*), 16 caractéristiques et une classe.

Les 16 caractéristiques sont :

1. *hair* : {Boolean} l'animal a des poils;
1. *feathers* :  {Boolean} si l'animal a des plumes;
1. *eggs* :  {Boolean} l'animal pond des œufs;
1. *milk* :  {Boolean} l'animal produit du lait;
1. *airborne* :  {Boolean} l'animal vole;
1. *aquatic* :  {Boolean} l'animal est aquatique;
1. *predator* :  {Boolean} l'animal est un prédateur;
1. *toothed* :  {Boolean} l'animal a des dents;
1. *backbone* :  {Boolean} l'animal a une colonne vertébrale;
1. *breathes* :  {Boolean} l'animal respire de l'air;
1. *venomous* :  {Boolean} l'animal est venimeux;
1. *fins* :  {Boolean} l'animal a des nageoires;
1. *legs* :  {Boolean} nombre de jambes, qui peut être: {0,2,4,5,6,8};
1. *tail* :  {Boolean} l'animal a une queue;
1. *domestic* :  {Boolean} l'animal est domestique;
1. *catsize* :  {Boolean} TBD;

Enfin la colonne *class_type* est un entier entre 1 et 7 correspondant à une entrée du fichier __class.csv__ (voir ci-dessous).

---

* __class.csv__ contient la descriptions des sept classes regroupant les animaux

In [ ]:
df_class = pd.read_csv("../input/zoo-animal-classification/class.csv")
df_class

In [ ]:
df_class.shape

Les septs classes sont ainsi :
1. _Mammal_ (Mammifère)
1. _Bird_ (Oiseau)
1. _Reptile_ (Reptile)
1. _Fish_ (Poisson)
1. _Amphibian_ (Amphibia)
1. _Bug_ (Insecte)
1. _Invertebrate_ (Invertébré)

---

Les colonnes *Number_Of_Animal_Species_In_Class* et *Animal_Names* sont obsolètes car facilement trouvables grâce au premier fichier et doivent être mises à jour pour être utilisées.

Par exemple pour la classe **1**, soit les mammifères.

Nombre de mammifères :

In [ ]:
len(df_zoo[df_zoo['class_type'] == 1].index)

Liste des mammifères :

In [ ]:
list(df_zoo[df_zoo['class_type'] == 1]['animal_name'])

In [ ]:
df_zoo2 = pd.read_csv("../input/zoo-animals-extended-dataset/zoo2.csv")
df_zoo3 = pd.read_csv("../input/zoo-animals-extended-dataset/zoo3.csv")

In [ ]:
df_zoo2.shape

In [ ]:
df_zoo3.shape

---

* __zoo2.csv__ et __zoo3.csv__ possède la même structure et les mêmes buts que __zoo.csv__, mais contiennent d'autres animaux.

## 1.3 Création du Dataframe avec tous les animaux

Avant de fusionner les jeux de données, on ajoute une nouvelle colonne avec le jeu de données de provenance de chaque animal pour garder une trace de cette information.

In [ ]:
df_zoo['id_zoo'] = 'zoo1'
df_zoo2['id_zoo'] = 'zoo2'
df_zoo3['id_zoo'] = 'zoo3'

On peut maintenant fusionner les trois Dataframe.

In [ ]:
zoo_complet = pd.concat([df_zoo, df_zoo2, df_zoo3], axis=0)

# 2. Exploration des données

## 2.1 Ajustement des types

In [ ]:
zoo_complet.dtypes

A ce stade, tous les descripteurs sont définis comme des entiers (_int64_). Il est plus cohérent pour l'analyse d'utiliser les types _category_ et _bool_.

In [ ]:
for col in ['hair', 'feathers', 'eggs', 'milk', 'airborne', 'aquatic', 'predator', 'toothed', 'backbone', 'breathes', 'venomous', 'fins', 'tail', 'domestic', 'catsize']:
    zoo_complet[col] = zoo_complet[col].astype('bool')
    
zoo_complet['legs'] = zoo_complet['legs'].astype('category')

In [ ]:
zoo_complet.dtypes

## 2.2 Quelques statistiques de présentation

In [ ]:
print ("Nombre d'animaux dans zoo.csv : %d" % df_zoo.shape[0])
print ("Nombre d'animaux dans zoo2.csv : %d" % df_zoo2.shape[0])
print ("Nombre d'animaux dans zoo3.csv : %d" % df_zoo3.shape[0])
print ("Nombre d'animaux dans la fusion : %d" % zoo_complet.shape[0])

Suite à cette fusion, on peut recalculer les colonnes *class_type* et *animal_name* du fichier **class.csv**

In [ ]:
ax = sns.countplot(x="class_type", data=zoo_complet) 

In [ ]:
ax = sns.countplot(x="class_type", hue="id_zoo", data=zoo_complet)         

## 2.3 Grouping By and Aggregation

Une question importante à se poser avant de commencer l'analyse est : "Toutes nos entrées ont-elles un ensemble de descripteurs distincts ?"

Si plusieurs entrées ont la même combinaison de valeurs, cela peut avoir deux conséquences :

1. Si ces entrées ont la même valeur de variable cible (ici la colonne *class_type*), alors cette combinaison de descripteurs prendra plus de poids dans notre apprentissage;
2. Si à l'inverse ces entrées ont des valeurs de variable cible différentes, alors cela veut dire qu'il ne sera pas possible d'atteindre un score de 100%. En effet, pour une même entrée nous allons demander au système de classification de prendre des décisions différentes.


In [ ]:
descriptors_columns = ['hair', 'feathers', 'eggs', 'milk', 'airborne', 'aquatic', 'predator', 'toothed', 'backbone', 'breathes', 'venomous', 'fins', 'tail', 'legs', 'domestic', 'catsize']

zoo_complet.groupby(descriptors_columns).filter(lambda g: (g['class_type'].nunique() > 1)).sort_values(by=descriptors_columns)

Nous remarquons qu'il existe 11 entrées partageant leur combinaison de valeurs avec au moins une autre entrée.

Plus précisement, trois combinaisons sont trouvées avec à chaque fois deux valeurs différentes pour **class_type**:
1. Celle partagée par _flea_, _termite_, _cricket_, _beetle_, _cockroach_ et _aphid_ (où **class_type** vaut 6, soit des **insectes**) et _bicho-pau_ (où **class_type** vaut 7, soit un **invertébré**);
2. Celle partagée par _trajaca_ (**class_type**=3; **Reptile**) et _sapo-barriga-de-fogo_ (**class_type**=5; **Amphibien**);
2. Celle partagée par _newt_ (**class_type**=5; **Amphibien**) et _jacare-coroa_ (**class_type**=3; **Reptile**).

## 2.4 Visualizing Data

Répartition des sept classes dans l'ensemble des données :

In [ ]:
classes = ["Mammifère", "Oiseau", "Reptile", "Poisson", "Amphibia", "Insecte", "Invertébré"]

In [ ]:
zoo_complet['class_type'].value_counts().sort_index()

In [ ]:
ax = sns.countplot(x="class_type", data=zoo_complet)
ax.set_xticklabels(classes, rotation=45)
plt.show()

Il est intéressant de regarder ce que cela donne au niveau de la répartition des sept classes dans les trois ensembles de données.
Il est possible de représenter cette répartition de deux façons selon si l'on souhaite mettre l'accent sur la répartition entre les trois jeux de données ou entre les familles d'espèces.

In [ ]:
ax = sns.countplot(x="id_zoo", hue="class_type", data=zoo_complet)
ax.set_xticklabels(["Zoo #1", "Zoo #2", "Zoo #3"], rotation=45)
plt.legend(title='Légende', ncol=2, labels=classes)
plt.show()

In [ ]:
ax = sns.countplot(x="class_type", hue="id_zoo", data=zoo_complet)
ax.set_xticklabels(classes, rotation=45)
plt.legend(title='Légende', labels=["Zoo #1", "Zoo #2", "Zoo #3"])
plt.show()

Distribution des valeurs dans chaque colonne :

In [ ]:
plt.subplots_adjust(left=0,
                    bottom=0,
                    right=2,
                    top=2,
                    wspace=0.35, 
                    hspace=0.35)
    
    
cpt = 0
for col in descriptors_columns:
    cpt += 1
    fig = plt.subplot(4,4,cpt)
    ax = sns.countplot(x=col, hue="class_type", data=zoo_complet)
    plt.legend(title='Légende', ncol=2, labels=classes)
plt.show()

